In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 
showHeader('Raw Disclosures',line2='Sixty days of FracFocus changes',use_remote=True)

In [ ]:
import datetime
today = datetime.datetime.today()
edate = today - datetime.timedelta(days=60)
print('earlist shown',edate)

In [ ]:
import os
import pickle
import openFF.common.text_handlers as th
arc_dir = r"C:\MyDocs\integrated\openFF_archive\diff_dicts"

diff_fns = os.listdir(arc_dir)
download_dates = []
added = []
changed = []
removed = []

for fn in diff_fns:
    tdate = datetime.datetime(int(fn[10:14]),int(fn[15:17]),int(fn[18:20]))
    # print(tdate)
    if tdate>= edate:
        download_dates.append(fn[10:20])
        with open(os.path.join(arc_dir,fn),'rb') as f:
            diff_dic = pickle.load(f)
        if len(diff_dic['removed_disc'])>0:
            t = diff_dic['removed_disc'].copy()
            # print(t.head())
            t['date_changed'] = tdate
            t['change_type'] = 'removed'
            removed.append(t)
        if len(diff_dic['added_disc'])>0:
            t = diff_dic['added_disc'].copy()
            t['date_changed'] = tdate
            t['change_type'] = 'added'
            added.append(t)
        if len(diff_dic['changed_disc'])>0:
            t = diff_dic['changed_disc'].copy()
            t['date_changed'] = tdate
            t['change_type'] = 'modified'            
            changed.append(t)
alllists = added + changed + removed
wholeset = pd.concat(alllists,sort=True)
# wholeset

## Download dates used

In [ ]:
print(download_dates)

In [ ]:
wholeset['job_end_date'] = wholeset.JobEndDate.str.split().str[0]
wholeset['job_end_date'] = pd.to_datetime(wholeset.job_end_date,format="%m/%d/%Y")
wholeset['FF_disc'] = wholeset.apply(lambda x: th.getFFLink(x), axis=1)
wholeset['disc_link'] = wholeset.apply(lambda x: th.getDisclosureLink(APINumber=x.APINumber,
                                                                      disclosureid=x.discID,
                                                                      text_to_show='Open-FF disclosure',
                                                                      use_remote=True,
                                                                      check_if_exists=False), axis=1)
wholeset['TBWV'] = wholeset.TotalBaseWaterVolume.astype('float64')
wholeset['TBWV'] = wholeset.TBWV.map(lambda x: th.round_sig(x,5))


Note that "disc_link" to the Open-FF disclosures will only be valid for those disclosures that have been through the curating process, that is, where 'date_changed' is before the bulk download date of the online repository.  See the header of this webpage for that date.

In [ ]:
wholeset[['FF_disc','disc_link','job_end_date','StateName','CountyName','OperatorName','TBWV','change_type','date_changed']]

In [ ]:
wholeset.change_type.value_counts()

In [ ]:
t = wholeset[wholeset.change_type=='modified'].date_changed.value_counts()
t = t.reset_index().sort_values('date_changed')
t.plot('date_changed','count',style='o')

In [ ]:
wholeset[wholeset.change_type=='changed'].date_changed.value_counts()